This is the first Kernel that I have ever published on Kaggle. Took me 3 months of learning to finally pluck up the courage and post my intital work on this reputed site. I really hope the Kernel helps novice Data Scientists/Machine Learning engineers like me understand the basic concepts of Machine Learning. Feedback would be HIGHLY appreciated, and would help me become a better coder :) Thank you for taking the time out in reading this.

I have used the following algortihms and calculated the AUC and accuracy for each of the Supervised learning algos. I am still a noob in the Unsupervised learning section, but I feel I'm making progress! :) 
Supevised Learning - Logistic Regression, Naive Bayes, Decision Trees, Random Forests
Unsupervised Learning - Principal Component Analysis (PCA), KMeans (Flat Clustering), MeanShift (Hierarchical Clustering)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data = pd.read_csv("../input/mushrooms.csv")

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
#ALl the variables are in string format. Convert categorical variables to integer using label encoder
from sklearn.preprocessing import LabelEncoder
lbl = LabelEncoder()

In [ ]:
for col in data.columns:
    data[col]=lbl.fit_transform(data[col])    

In [ ]:
#check the data after label encoding
data.head()

In [ ]:
#split the x and y variables
y=data['class']
x=data.iloc[:,1:23]

In [ ]:
#check shape of new variables
x.shape

In [ ]:
y.shape

In [ ]:
#check data
x.head

In [ ]:
y.head

In [ ]:
#I want to use PCA on this data. First normalise the data using StandardScalar so that the data is now between -1 and 1
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x = sc.fit_transform(x)

In [ ]:
#see the Standardised data
print(x)

In [ ]:
#using principal component analysis
#Even though the number of variables is not too high, I would still like to use PCA to see which variables describe the maximum variance in data
from sklearn.decomposition import PCA
pca = PCA()
x_pca = pca.fit_transform(x)

In [ ]:
#plot a Scree plot of the Principal Components
plt.figure(figsize=(16,11))
plt.plot(np.cumsum(pca.explained_variance_ratio_), 'ro-')
plt.grid()

In [ ]:
#from the graph, first 17 components describe the maximum variance(more than 90% of the data). We shall use them for our subsequent analysis.
new_pca = PCA(n_components=17)

In [ ]:
x_new = new_pca.fit_transform(x)

In [ ]:
#using KMeans to plot the clusters. We know that we habe 2 classes of the target variable. So n_clusters=2
from sklearn.cluster import KMeans
k_means = KMeans(n_clusters=2)

In [ ]:
k_means.fit_predict(x_new )

In [ ]:
#plot the clusters.
colors = ['r','g']
for i in range(len(x_new)):
    plt.scatter(x_new[i][0], x_new[i][1], c=colors[k_means.labels_[i]], s=10)
plt.show()

In [ ]:
#2 distinct clusters are created. Data points are far apart 
x_new.shape

In [ ]:
#separate the train and test data
from sklearn.cross_validation import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_new, y, test_size = 0.25, random_state = 6)

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
#using Logistic regression to build the first model
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train, y_train)
lr_predict =lr.predict(x_test)

In [ ]:
lr_predict_prob = lr.predict_proba(x_test)

In [ ]:
print(lr_predict)
print(lr_predict_prob[:,1])

In [ ]:
#import metrics
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
lr_conf_matrix = confusion_matrix(y_test, lr_predict)
lr_accuracy = accuracy_score(y_test, lr_predict)

In [ ]:
print(lr_conf_matrix)
print(lr_accuracy)

In [ ]:
#roc curve
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_test,lr_predict_prob[:,1] )

In [ ]:
#auc score
from sklearn.metrics import auc
lr_auc = auc(fpr, tpr)
print(lr_auc)

In [ ]:
#plotting ROC curve
plt.figure(figsize=(10,9))
plt.plot(fpr, tpr, label = 'AUC= %0.2f' % lr_auc )
plt.plot([0,1],[0,1], linestyle = '--')
plt.legend()

In [ ]:
#Using Naive Bayes
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(x_train, y_train)
gnb_predict = gnb.predict(x_test)
gnb_predict_prob = gnb.predict_proba(x_test)

In [ ]:
print(gnb_predict)
print(gnb_predict_prob)

In [ ]:
gnb_conf_matrix = confusion_matrix(y_test, gnb_predict)
gnb_accuracy_score = accuracy_score(y_test, gnb_predict)

In [ ]:
print(gnb_conf_matrix)
print(gnb_accuracy_score)

In [ ]:
#calculate ROC and AUC
fpr, tpr, thresholds = roc_curve(y_test, gnb_predict_prob[:,1])
#print auc
gnb_auc = auc(fpr, tpr)
print(gnb_auc)

In [ ]:
#plot ROC curve
plt.figure(figsize=(10,9))
plt.plot(fpr, tpr, label = 'AUC %0.2f' % gnb_auc)
plt.plot([0,1],[0,1], linestyle = '--')
plt.legend()

In [ ]:
#lets use Decision Trees to classify 
#use the number of trees as 10 first
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(max_depth=10)

In [ ]:
dt.fit(x_train,y_train)
dt_predict = dt.predict(x_test)
dt_predict_prob = dt.predict_proba(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
dt_conf_matrix = confusion_matrix(y_test, dt_predict)
dt_accuracy_score = accuracy_score(y_test, dt_predict)

In [ ]:
print(dt_conf_matrix)
print(dt_accuracy_score)

In [ ]:
#calculate auc and plot roc
from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(y_test, dt_predict_prob[:,1])
dt_auc = auc(fpr, tpr)
print(dt_auc)

In [ ]:
#plot ROC curve
plt.figure(figsize=(10,9))
plt.plot(fpr, tpr, label = 'AUC %0.2f' % dt_auc)
plt.plot([0,1],[0,1], linestyle = '--')
plt.xlabel('False Positive rate')
plt.ylabel('True Positive rate')
plt.legend()
plt.grid()


In [ ]:
#using random forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth=10) #10 trees
rf.fit(x_train, y_train)
rf_predict = rf.predict(x_test)
rf_predict_prob = rf.predict_proba(x_test)

In [ ]:
rf_conf_matrix = confusion_matrix(y_test,rf_predict)
rf_accuracy_score = accuracy_score(y_test, rf_predict)

In [ ]:
print(rf_conf_matrix)
print(rf_accuracy_score)
#random forest has a higher accuracy score than the decision tree
#Decision tree = 99.3
#Random forest = 99.9

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, rf_predict_prob[:,1])
rf_auc = auc(fpr, tpr)
print(rf_auc)

In [ ]:
#plot the ROC curve
plt.figure(figsize=(10,9))
plt.plot(fpr, tpr, label = 'AUC: %0.2f' % rf_auc)
plt.plot([1,0],[1,0], linestyle = '--')
plt.legend(loc=0)
plt.xlabel('False Positive rate')
plt.ylabel('True Positive rate')
plt.grid()

Let's try some Hierarchical unsupervised learning technique to see how many clusters the algorithm predicts.
I will be using MeanShift algorithm.
PS - This is just for knowledge's sake - a method that I am utilising to help me understand basic machine learning techniques.

In [ ]:
#how would an unsupervised algo like MeanShift or DBScan work? Let's find out
from sklearn.cluster import MeanShift
ms = MeanShift()
ms.fit(x_new)

In [ ]:
#print the labels and the cluster centers (I will be calling them centroids)
ms_labels = ms.labels_
ms_centroids = ms.cluster_centers_
print(ms_labels)
print(ms_centroids)

In [ ]:
#np.unique will give us one count of each label. 
n_clusters = len(np.unique(ms_labels))
print(n_clusters)

hmmm so unsupervised learning with MeanShift gives us 3 clusters! Interesting!

In [ ]:
#let's plot the clusters and see how different they are from our original cluster of KMeans'
plt.figure(figsize=(10,9))
colors = ['r','g','y','b']
for i in range(len(x_new)):
    plt.scatter(x_new[i][0], x_new[i][1], c=colors[ms_labels[i]], s=5)
#print cluster centers
#Cluster centers are x's in blue
plt.scatter(ms_centroids[:,0], ms_centroids[:,1], marker='x')
plt.show()
#Considerably different!